In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import time

In [2]:
# Pandas'a özel parametreler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 3)
pd.set_option('display.width', 500)
pd.set_option("display.float_format", lambda x: '%.2f' % x)


In [3]:
df = pd.read_csv('MTA_Subway.csv')

C:\Users\ahmet\AppData\Local\Temp\ipykernel_6908\2522875168.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MTA_Subway.csv')


In [4]:
df.dtypes

transit_timestamp      object
station_complex_id     object
station_complex        object
borough                object
routes                 object
payment_method         object
ridership               int64
transfers               int64
latitude              float64
longitude             float64
Georeference           object
itsuid                 object
dtype: object

In [5]:
df.columns

Index(['transit_timestamp', 'station_complex_id', 'station_complex', 'borough', 'routes', 'payment_method', 'ridership', 'transfers', 'latitude', 'longitude', 'Georeference', 'itsuid'], dtype='object')

In [6]:
df.head()

,transit_timestamp,station_complex_id,station_complex,borough,routes,payment_method,ridership,transfers,latitude,longitude,Georeference,itsuid
0,03/27/2022 06:00:00 AM,H007,1 Av (L),M,L,omny,12,0,40.731,-73.982,POINT (-73.98162841796875 40.730953216552734),2022-03-27T06:00:00H0071 Av (L)MLomny
1,06/05/2022 02:00:00 AM,H007,1 Av (L),M,L,omny,93,0,40.731,-73.982,POINT (-73.98162841796875 40.730953216552734),2022-06-05T02:00:00H0071 Av (L)MLomny
2,06/13/2023 03:00:00 AM,H007,1 Av (L),M,L,omny,3,0,40.731,-73.982,POINT (-73.98162841796875 40.730953216552734),2023-06-13T03:00:00H0071 Av (L)MLomny
3,07/28/2023 05:00:00 AM,H007,1 Av (L),M,L,omny,31,0,40.731,-73.982,POINT (-73.98162841796875 40.730953216552734),2023-07-28T05:00:00H0071 Av (L)MLomny
4,08/08/2023 03:00:00 AM,H007,1 Av (L),M,L,omny,3,0,40.731,-73.982,POINT (-73.98162841796875 40.730953216552734),2023-08-08T03:00:00H0071 Av (L)MLomny


In [7]:
df = df.drop(columns=['station_complex', 'routes', 'payment_method', 'transfers', 'Georeference','itsuid'])

In [9]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [10]:
df['borough'].replace({'M': 'Manhattan', 'Q': 'Queens', 'BX': 'Bronx', 'BK': 'Brooklyn', 'SI': 'Staten Island'}, inplace=True)

In [11]:
# transit_timestamp sütununu datetime türüne çeviriyoruz
df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])

# Tarih ve saat sütunlarını ayrı ayrı oluşturuyoruz
df['date'] = df['transit_timestamp'].dt.date
df['hour'] = df['transit_timestamp'].dt.time

In [12]:
df['date'] = pd.to_datetime(df['date'])

df['day_of_week'] = df['date'].dt.strftime('%A')

In [13]:
df_grouped2 = df.groupby('day_of_week')['ridership'].sum().reset_index().sort_values(by='ridership', ascending=False)
print(df_grouped2)

  day_of_week  ridership
6   Wednesday  305593582
4    Thursday  302888637
5     Tuesday  297248767
0      Friday  283701101
1      Monday  257292520
2    Saturday  189043914
3      Sunday  150438140


In [ ]:
df.head()

In [14]:
df.to_csv('transformed_mta_data.csv', index=False)